## 5.2 填充和步幅

一般来说，假设输入形状是$n_h\times n_w$，卷积核窗口形状是$k_h\times k_w$，那么输出形状将会是
$$(n_h-k_h+1) \times (n_w-k_w+1).$$

所以卷积层的输出形状由输入形状和卷积核窗口形状决定。

### 5.2.1 填充

- 填充(padding)是指在输入高和宽的两侧填充元素(通常是0元素)。
- 一般来说，如果在高的两侧一共填充$p_h$行，在宽的两侧一共填充$p_w$列，那么输出形状将会是
$$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1),$$
也就是说，输出的高和宽会分别增加$p_h$和$p_w$。
- 在很多情况下，我们会设置$p_h=k_h-1$和$p_w=k_w-1$来使输入和输出具有相同的高和宽。这样会方便在构造网络时推测每个层的输出形状。
- 假设这里$k_h$是奇数，我们会在高的两侧分别填充$p_h/2$行。
- 如果$k_h$是偶数，一种可能是在输入的顶端一侧填充$\lceil p_h/2\rceil$行，而在底端一侧填充$\lfloor p_h/2\rfloor$行。
- 卷积神经网络经常使用奇数高和宽的卷积核，如1、3、5和7，所以两端上的填充个数相等。

In [6]:
from mxnet import nd
from mxnet.gluon import nn

# 定义一个函数来计算卷积层。它初始化卷积层权重，并对输入和输出做相应的升维和将维
def comp_conv2d(conv2d, X):
    conv2d.initialize()
    # (1, 1)代表批量大小和通道数均为1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:]) # 排除不关心的前两维：批量和通道

In [5]:
# 注意这里是两侧分别填充1行或列，所以在两侧一共填充2行或列
conv2d = nn.Conv2D(1, kernel_size=3, padding=1)
X = nd.random_uniform(shape=(8, 8))
comp_conv2d(conv2d, X).shape

(8, 8)

当卷积核的高和宽不同时，我们也可以通过设置高和宽上不同的填充数使输出和输入具有相同的高和宽。

In [7]:
# 使用高为5、宽为3的卷积核。在高和宽两侧的填充数分别为2和1。
conv2d = nn.Conv2D(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

(8, 8)

### 5.2.2 步幅

- 我们将每次滑动的行数和列数称为步幅(stride)。
- 一般来说，当高上步幅为$s_h$，宽上步幅为$s_w$时，输出形状为
$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$
- 如果设置$p_h=k_h-1$和$p_w=k_w-1$，那么输出形状将简化为
$$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$$
- 更进一步，如果输入的高和宽能分别被高和宽上的步幅整除，那么输出形状将是
$$(n_h/s_h) \times (n_w/s_w)$$

In [8]:
conv2d = nn.Conv2D(1, kernel_size=3, padding=1, strides=2)
comp_conv2d(conv2d, X).shape

(4, 4)

In [9]:
conv2d = nn.Conv2D(1, kernel_size=(3, 5), padding=(0, 1), strides=(3, 4))
comp_conv2d(conv2d, X).shape

(2, 2)

- 为了表述简洁，当输入的高和宽两侧的填充数分别为$p_h$和$p_w$时，我们称填充为$(p_h, p_w)$。特别地，当$p_h = p_w = p$时，填充为$p$。
- 当在高和宽上的步幅分别为$s_h$和$s_w$时，我们称步幅为$(s_h, s_w)$。特别地，当$s_h = s_w = s$时，步幅为$s$。
- 在默认情况下，填充为0，步幅为1。

### 小结

- 填充可以增加输出的高和宽。这常用来使输出与输入具有相同的高和宽。
- 步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的$1/n$($n$为大于1的整数)。